## Sum of Square Errors Illustration
This is an example of using **`ipywidgets`** in conjuction with **`bokeh`**.  The plot illustrates interactively fitting a curve to some data points by minimizing the sum of the squared errors.  Of course, there are better ways to fit a function to a bunch of points than as done here.  The point of this notebook is to illustrate using widgets and bokeh, it is not how to best fit a curve to data points.

#### Import modules

In [ ]:
import numpy as np
from ipywidgets import interact, widgets
from bokeh.plotting import figure
from bokeh.io import show, output_notebook, push_notebook

output_notebook()

#### (Pretend you don't see this fake data being generated)

In [ ]:
np.random.seed(1)  # seed value of 1 better samples lower x range
N = 30
lo = 10.
hi = 70.
x = lo + (hi - lo) * np.random.random(N)
x = np.sort(x)

scale_fac = 8.7
pow_index = 1.7
y = scale_fac * x**(-pow_index)
dy = 1. + 0.3*np.random.randn(len(y))
y *= dy

#### We have some $(x,y)$ data points which we think have some relationship $y(x)$

In [ ]:
f = figure(plot_height=300, plot_width=800, sizing_mode='scale_width')
f.circle(x=x, y=y, size=5, color='blue')
show(f)

#### A Simple Model
We want a simple model for the relationship, something we can hopefully use to predict $y$ decently, given a new $x$.  A power law is a simple model which looks appropriate here:

$$
y = A \cdot x^{-p}
$$

We want to find values of the scaling factor $A$ and the power law index $p$.  

In [ ]:
# initial guesses for scale factor and power law index
A0 = 0.1
p0 = 0.0

yfit = A0 * x**(-p0)
min_err_found = np.Infinity
best_A = A0
best_p = p0

f = figure(plot_height=300, plot_width=800, sizing_mode='scale_width')

# add fit line and errors
yline = f.line(x=x, y=yfit, color='red', line_alpha=0.75, line_width=1.5, legend='best fit')
ysegs = f.segment(x0=x, x1=x, y0=y, y1=yfit, color='red', line_alpha=0.75, line_width=1.5)

f.circle(x=x, y=y, size=5, color='blue')
show(f, notebook_handle=True)

def update(const, index):
    yf = const * x**(-index)
    yline.data_source.data['y'] = yf
    ysegs.data_source.data['y1'] = yf

    sum_sq_err = np.sum(np.abs(y-yf)**2)
    global min_err_found
    global best_A
    global best_p
    if sum_sq_err < min_err_found:
        min_err_found = sum_sq_err
        best_A = const
        best_p = index
    
    title_text = 'Sum sq. err = '+str(np.around(sum_sq_err, decimals=5))
    title_text += ', lowest yet = '+str(np.around(min_err_found, decimals=5))
    title_text += ' for A = '+str(best_A)+', p = '+str(best_p)
    
    f.title.text = title_text
    f.title.align = "left"
    push_notebook()


const = widgets.FloatSlider(
            value=A0,
            min=0.1,
            max=20.,
            step=0.1,
            description='A:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )

index = widgets.FloatSlider(
            value=p0,
            min=-1.,
            max=3.,
            step=0.1,
            description='p:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
        )

interact(update, const=const, index=index)